In [ ]:
# Import findspark and initialize.
import findspark
findspark.init()


In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [1]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)


NameError: name 'spark' is not defined

In [3]:
# 2. Create a temporary view of the DataFrame.
df.show()

NameError: name 'df' is not defined

In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
prices=spark.sql('Select Year(date) as Year, Round(Avg(price),2) as average_price from home_sales where bedrooms=4 group by Year')
prices.show()

In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
prices_3_bed_3_bath=spark.sql('Select date_built, Round(Mean(price),2) As average_price from home_sales where bedrooms=3 and bathrooms=3 group by date_built')
prices_3_bed_3_bath.show()

In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
prices_3_bed_3_bath_2_floors_2000_sq_feet=spark.sql('Select date_built, Round(Mean(price),2) As average_price from home_sales where bedrooms=3 and bathrooms=3 and floors=2 and sqft_living>=2000 group by date_built')
prices_3_bed_3_bath_2_floors_2000_sq_feet.show()

In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

price_per_view=spark.sql('Select view, Round(avg(price),2) As average_price from home_sales group by view having average_price>=350000 order by view desc')
price_per_view.show()

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
spark.sql('Select view, Round(Avg(price),2) As average_price from home_sales group by view having average_price>=350000 order by view desc').show()



print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
spark.sql('Select * from home_sales').write.partitionBy('date_built').parquet("home_sales_partitioned", mode="overwrite")


In [ ]:
# 11. Read the formatted parquet data.
parquet_data=spark.read.parquet('home_sales_partitioned')


In [ ]:
# 12. Create a temporary table for the parquet data.
parquet_data.createOrReplaceTempView('parquet_temp')


In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()


spark.sql('Select view, Round(Avg(price),2) As average_price from parquet_temp group by view having average_price>=350000 order by view desc').show()

print("--- %s seconds ---" % (time.time() - start_time))

#the runtime is shorter with the parquet

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')
